###Team Member : Raj Khatri, Lokendra Sinha

###Topic : Travel Itinerary Planner using Multi-AI agents

###Problem we are solving :

####1.Time-Saving ⏳

Instantly generates complete itineraries (vs. hours of manual research)

####2.Personalization 🎯

Tailors plans to your interests (food, adventure, culture etc.)

Learns your preferences over time (via User ID)

####3.Smart Recommendations 🤖

Combines:

AI itinerary generation (LLM+Gen-AI+Agentic-AI)

Personalized suggestions (ML)

Review analysis (DL)

####4.Local Expertise 🗺️

Reveals hidden gems & authentic experiences

Avoids tourist traps

####5.Decision Simplicity ✅

Curated daily plans with optimal timing

Clear sentiment analysis of locations


###Key Benefit: Turns stressful trip planning into one-click perfect itineraries.

 Install Required Packages

In [1]:
!pip install langchain langchain_core langchain_groq langchain_community langgraph gradio
!pip install scikit-learn transformers torch pandas

Importing Libraries

In [2]:
import os
from typing import TypedDict, Annotated, List
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
import gradio as gr
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
from transformers import pipeline

Defining the State Structure

In [3]:
class PlannerState(TypedDict):
    """Structure of our travel planning state"""
    messages: Annotated[List[HumanMessage | AIMessage], "The messages in the conversation"]
    city: str
    interests: List[str]
    itinerary: str
    user_id: int

Initializing the LLM (Groq API)

In [ ]:
# Initialize the Groq LLM (replace with your actual API key)
llm = ChatGroq(
    temperature=0,
    groq_api_key="YOUR_GROQ_API",
    model_name="llama-3.3-70b-versatile"
)

Defining Prompt Templates

In [5]:
# Main itinerary prompt
itinerary_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a helpful travel assistant. Create a detailed day trip itinerary for {city}
    based on the user's interests: {interests}. Provide a well-structured itinerary with:
    - Morning, afternoon, and evening activities
    - Recommended restaurants/cafes
    - Travel tips
    - Estimated times for each activity"""),
    ("human", "Create an itinerary for my day trip to {city} with interests in {interests}."),
])

# Follow-up prompt for additional recommendations
recommendation_prompt = ChatPromptTemplate.from_messages([
    ("system", """Based on the user's interests in {interests} and their current itinerary:
    {itinerary}
    Suggest 2-3 additional activities they might enjoy in {city}."""),
    ("human", "Suggest some additional activities I might like."),
])

Machine Learning Personalized Recommendation Model (Random Forest)

In [6]:
# Mock dataset for training our recommendation model
def create_mock_data():
    data = {
        "user_id": [1, 1, 1, 2, 2, 3, 3, 3, 4, 4],
        "activity": ["beach", "museum", "hiking", "shopping", "beach",
                    "museum", "restaurant", "park", "hiking", "shopping"],
        "liked": [1, 0, 1, 1, 0, 1, 1, 0, 1, 0]
    }
    df = pd.DataFrame(data)

    # Encode activities
    activity_map = {activity: idx for idx, activity in enumerate(df['activity'].unique())}
    df['activity_encoded'] = df['activity'].map(activity_map)

    return df, activity_map

# Train the model
df, activity_map = create_mock_data()
model = RandomForestClassifier()
model.fit(df[["user_id", "activity_encoded"]], df["liked"])

def recommend_activity(user_id: int, activities: List[str]) -> List[str]:
    """Predict which activities a user will like"""
    # Encode the input activities
    encoded_activities = [activity_map[a] for a in activities if a in activity_map]

    if not encoded_activities:
        return []

    # Get predictions
    predictions = model.predict([[user_id, a] for a in encoded_activities])

    # Return only recommended activities
    return [a for a, p in zip(activities, predictions) if p == 1]

Using DL for Sentiment Analysis Model (BERT)

In [7]:
# Initialize sentiment analyzer
sentiment_analyzer = pipeline(
    "sentiment-analysis",
    model="distilbert-base-uncased-finetuned-sst-2-english"
)

def analyze_reviews(reviews: List[str]) -> str:
    """Analyze sentiment of location reviews"""
    if not reviews:
        return "No reviews available for sentiment analysis."

    results = sentiment_analyzer(reviews)
    positive = sum(1 for r in results if r['label'] == 'POSITIVE')
    percentage = (positive / len(results)) * 100

    sentiment = "positive" if percentage > 60 else "negative" if percentage < 40 else "mixed"
    return f"Sentiment analysis of {len(results)} reviews: {percentage:.1f}% positive ({sentiment} sentiment)"

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cpu


Defining Agent Functions

In [8]:
def input_city(state: PlannerState) -> PlannerState:
    """Agent to handle city input"""
    city = state['city']  # Get city from the state
    return {
        **state,
        "messages": state['messages'] + [HumanMessage(content=f"Planning trip to: {city}")],
    }

def input_interests(state: PlannerState) -> PlannerState:
    """Agent to handle interests input"""
    interests_list = [i.strip() for i in state['interests']]
    return {
        **state,
        "messages": state['messages'] + [HumanMessage(content=f"My interests: {', '.join(interests_list)}")],
    }

def create_itinerary(state: PlannerState) -> PlannerState:
    """Generate the base itinerary using LLM"""
    # Generate initial itinerary
    itinerary = llm.invoke(
        itinerary_prompt.format_messages(
            city=state['city'],
            interests=", ".join(state['interests'])
        )
    ).content

    # Add ML/DL enhancements
    enhancements = []

    # 1. Personalized recommendations
    if len(state['interests']) > 1:
        recommended = recommend_activity(state['user_id'], state['interests'])
        if recommended:
            enhancements.append(f"\n\nPersonalized recommendations based on your profile: {', '.join(recommended)}")

    # 2. Sentiment analysis for locations
    if "beach" in state['interests']:
        beach_reviews = [
            "Beautiful sandy beach with clear water!",
            "Too crowded on weekends.",
            "Perfect for sunset views."
        ]
        sentiment = analyze_reviews(beach_reviews)
        enhancements.append(f"\n\nBeach sentiment analysis: {sentiment}")

    # Combine all enhancements
    enhanced_itinerary = itinerary + "".join(enhancements)

    return {
        **state,
        "itinerary": enhanced_itinerary,
        "messages": state['messages'] + [AIMessage(content=enhanced_itinerary)],
    }

Build the Agentic Workflow

In [9]:
# Initialize the workflow
workflow = StateGraph(PlannerState)

# Add nodes (agents)
workflow.add_node("input_city", input_city)
workflow.add_node("input_interests", input_interests)
workflow.add_node("create_itinerary", create_itinerary)

# Define the workflow
workflow.set_entry_point("input_city")
workflow.add_edge("input_city", "input_interests")
workflow.add_edge("input_interests", "create_itinerary")
workflow.add_edge("create_itinerary", END)

# Compile the workflow
app = workflow.compile()

Creating Gradio Interface

In [13]:
def travel_planner(city: str, interests: str, user_id: int = 1):
    """Main function to handle the travel planning"""
    try:
        # Initialize state with all required fields
        state = {
            "messages": [],
            "city": city.strip(),
            "interests": [i.strip() for i in interests.split(",")],
            "itinerary": "",
            "user_id": int(user_id)
        }

        # Validate inputs
        if not state['city']:
            return "⚠️ Please enter a city name"
        if not state['interests'] or all(not i for i in state['interests']):
            return "⚠️ Please enter at least one interest"

        # Run the workflow - now passing just the state
        result = app.invoke(state)

        # Rest of your function remains the same
        ...
        # Format the output as markdown
        output = f"""
#  Trip to {state['city'].title()}

**Interests:** {', '.join(state['interests'])}

---

## Your Personalized Itinerary

{result['itinerary']}

---
"""
        return output

    except Exception as e:
        return f"❌ An error occurred: {str(e)}"

# Create the interface
interface = gr.Interface(
    fn=travel_planner,
    inputs=[
        gr.Textbox(label="City", placeholder="e.g., Paris, Tokyo, New York"),
        gr.Textbox(label="Interests", placeholder="e.g., museums, food, history"),
        gr.Number(label="User ID (for personalization)", value=1)
    ],
    outputs=gr.Markdown(label="Travel Itinerary"),  # Changed to Markdown
    title="Travel Itinerary Planner",
    description="Get personalized travel itineraries powered by AI",
    examples=[
        ["Paris", "museums, food, history", 1],
        ["Tokyo", "shopping, temples, technology", 2],
        ["New York", "broadway, parks, shopping", 3]
    ],
    allow_flagging="never"
)

# Launch the interface
interface.launch(share=True)

/usr/local/lib/python3.11/dist-packages/gradio/interface.py:416: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4d0035b97dbf45de96.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
